In [1]:
# Cell 1: Imports + Loading Saved Datasets and Model
import tensorflow as tf
import numpy as np
import pickle
import os
from tqdm import tqdm

print("Training notebook initialized")
print(f"TensorFlow version: {tf.__version__}")

# Load preprocessed data
print("\nLoading datasets...")
with open('data/train_dataset.pkl', 'rb') as f:
    x_train = pickle.load(f)
with open('data/test_dataset.pkl', 'rb') as f:
    x_test = pickle.load(f)

print(f"Training samples: {len(x_train)}")
print(f"Test samples: {len(x_test)}")

# Convert to tf.data.Dataset
BATCH_SIZE = 128
train_dataset = tf.data.Dataset.from_tensor_slices(x_train)
train_dataset = train_dataset.shuffle(10000, seed=42).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

print("Dataset loaded and prepared for training")

Training notebook initialized
TensorFlow version: 2.12.1

Loading datasets...
Training samples: 60000
Test samples: 10000
Dataset loaded and prepared for training


In [2]:
# Cell 2: Custom EBM Model Class
print("\n" + "="*60)
print("Defining custom Contrastive Divergence training model...")

class ContrastiveEnergyNetwork(tf.keras.Model):
    """
    Custom Keras model implementing Energy-Based Model training via Contrastive Divergence.
    
    Contrastive Divergence (CD-k) Algorithm:
    1. Positive Phase: Compute energy of real data samples (should be low)
    2. Negative Phase: 
       - Start from real data
       - Run k steps of MCMC (Langevin dynamics) to sample from model
       - Compute energy of these model samples (should be high)
    3. Loss: Difference between positive and negative energies
    4. Gradient: Minimize this difference to push real data energy down
       and model sample energy up
    
    The energy function learns to assign low energy to real data and high energy
    to everything else, effectively learning the data distribution.
    """
    
    def __init__(self, energy_network, step_size=0.01, cd_steps=3, noise_scale=0.01, **kwargs):
        super(ContrastiveEnergyNetwork, self).__init__(**kwargs)
        self.energy_net = energy_network
        self.step_size = step_size  # Langevin dynamics step size
        self.cd_steps = cd_steps    # Number of CD steps (k)
        self.noise_scale = noise_scale  # Noise for Langevin dynamics
        
        # Metrics
        self.pos_energy_metric = tf.keras.metrics.Mean(name='positive_energy')
        self.neg_energy_metric = tf.keras.metrics.Mean(name='negative_energy')
        self.loss_metric = tf.keras.metrics.Mean(name='cd_loss')
    
    def langevin_step(self, samples):
        """
        One step of Langevin dynamics for MCMC sampling.
        
        x_{t+1} = x_t - step_size * grad_x E(x_t) + noise
        
        This moves samples toward lower energy regions while adding noise
        to explore the energy landscape.
        """
        with tf.GradientTape() as tape:
            tape.watch(samples)
            energies = self.energy_net(samples, training=True)
            # Sum energies for gradient computation
            total_energy = tf.reduce_sum(energies)
        
        # Compute gradient of energy w.r.t. input
        energy_grad = tape.gradient(total_energy, samples)
        
        # Langevin update: move against gradient + noise
        new_samples = samples - self.step_size * energy_grad + \
                     tf.random.normal(tf.shape(samples), stddev=self.noise_scale)
        
        # Clip to valid range [-1, 1]
        new_samples = tf.clip_by_value(new_samples, -1.0, 1.0)
        
        return new_samples
    
    def contrastive_divergence_sampling(self, real_batch):
        """
        Perform CD-k: start from real data and run k MCMC steps.
        """
        # Initialize from real data (positive phase starting point)
        current_samples = tf.identity(real_batch)
        
        # Run k steps of Langevin dynamics
        for _ in range(self.cd_steps):
            current_samples = self.langevin_step(current_samples)
        
        return current_samples
    
    def train_step(self, real_batch):
        """
        Custom training step implementing Contrastive Divergence.
        """
        with tf.GradientTape() as tape:
            # Positive phase: energy of real data
            pos_energies = self.energy_net(real_batch, training=True)
            pos_energy_mean = tf.reduce_mean(pos_energies)
            
            # Negative phase: sample from model using CD-k
            model_samples = self.contrastive_divergence_sampling(real_batch)
            neg_energies = self.energy_net(model_samples, training=True)
            neg_energy_mean = tf.reduce_mean(neg_energies)
            
            # Contrastive Divergence loss: difference between phases
            # We want to minimize positive energy and maximize negative energy
            cd_loss = pos_energy_mean - neg_energy_mean
        
        # Compute gradients
        trainable_vars = self.energy_net.trainable_variables
        gradients = tape.gradient(cd_loss, trainable_vars)
        
        # Apply gradients
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        
        # Update metrics
        self.pos_energy_metric.update_state(pos_energy_mean)
        self.neg_energy_metric.update_state(neg_energy_mean)
        self.loss_metric.update_state(cd_loss)
        
        return {
            'loss': self.loss_metric.result(),
            'pos_energy': self.pos_energy_metric.result(),
            'neg_energy': self.neg_energy_metric.result()
        }
    
    def call(self, inputs):
        """Forward pass through energy network."""
        return self.energy_net(inputs)

# Build energy network (same as notebook 02)
def build_energy_network(input_dim=784):
    inputs = tf.keras.Input(shape=(input_dim,), name='image_input')
    hidden1 = tf.keras.layers.Dense(512, activation='relu', name='energy_hidden_1',
                                   kernel_initializer=tf.keras.initializers.GlorotUniform(seed=42))(inputs)
    hidden2 = tf.keras.layers.Dense(256, activation='relu', name='energy_hidden_2',
                                   kernel_initializer=tf.keras.initializers.GlorotUniform(seed=42))(hidden1)
    hidden3 = tf.keras.layers.Dense(128, activation='relu', name='energy_hidden_3',
                                   kernel_initializer=tf.keras.initializers.GlorotUniform(seed=42))(hidden2)
    energy_output = tf.keras.layers.Dense(1, activation=None, name='energy_output',
                                         kernel_initializer=tf.keras.initializers.GlorotUniform(seed=42))(hidden3)
    model = tf.keras.Model(inputs=inputs, outputs=energy_output, name='energy_function_network')
    return model

# Create model instance
energy_net = build_energy_network()
cd_model = ContrastiveEnergyNetwork(
    energy_network=energy_net,
    step_size=0.01,      # Langevin step size
    cd_steps=3,          # CD-k with k=3 (balanced quality vs speed)
    noise_scale=0.01     # Noise for exploration
)

# Compile with optimizer
cd_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001))

print(f"CD-k steps: {cd_model.cd_steps}")
print(f"Langevin step size: {cd_model.step_size}")
print("Custom model created successfully")


Defining custom Contrastive Divergence training model...
CD-k steps: 3
Langevin step size: 0.01
Custom model created successfully


In [3]:
# Cell 3: Training Loop
print("\n" + "="*60)
print("Starting training for 120 epochs...")

NUM_EPOCHS = 120
PRINT_FREQ = 10  # Print every N batches

# Storage for loss history
training_history = {
    'epoch': [],
    'loss': [],
    'pos_energy': [],
    'neg_energy': []
}

print(f"\nTraining configuration:")
print(f"  Epochs: {NUM_EPOCHS}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Learning rate: 0.0001")
print(f"  CD-k steps: {cd_model.cd_steps}")
print("\n" + "-"*60)

# Training loop
for epoch in range(1, NUM_EPOCHS + 1):
    # Reset metrics
    cd_model.pos_energy_metric.reset_states()
    cd_model.neg_energy_metric.reset_states()
    cd_model.loss_metric.reset_states()
    
    batch_count = 0
    epoch_losses = []
    epoch_pos_energies = []
    epoch_neg_energies = []
    
    for batch_idx, real_batch in enumerate(train_dataset):
        batch_count += 1
        
        # Train step
        metrics = cd_model.train_step(real_batch)
        
        epoch_losses.append(metrics['loss'].numpy())
        epoch_pos_energies.append(metrics['pos_energy'].numpy())
        epoch_neg_energies.append(metrics['neg_energy'].numpy())
        
        # Print batch-level info periodically
        if (batch_idx + 1) % PRINT_FREQ == 0:
            print(f"Epoch {epoch:3d} | Batch {batch_idx+1:4d} | "
                  f"Positive Energy: {metrics['pos_energy']:.4f} | "
                  f"Negative Energy: {metrics['neg_energy']:.4f}")
    
    # Epoch summary
    mean_loss = np.mean(epoch_losses)
    mean_pos = np.mean(epoch_pos_energies)
    mean_neg = np.mean(epoch_neg_energies)
    
    training_history['epoch'].append(epoch)
    training_history['loss'].append(mean_loss)
    training_history['pos_energy'].append(mean_pos)
    training_history['neg_energy'].append(mean_neg)
    
    print(f"\nEpoch {epoch:3d} completed | Mean loss = {mean_loss:.6f} | "
          f"Pos Energy = {mean_pos:.4f} | Neg Energy = {mean_neg:.4f}")
    print("-"*60)

print(f"\n✓ Training completed after {NUM_EPOCHS} epochs")


Starting training for 120 epochs...

Training configuration:
  Epochs: 120
  Batch size: 128
  Learning rate: 0.0001
  CD-k steps: 3

------------------------------------------------------------
Epoch   1 | Batch   10 | Positive Energy: -0.4988 | Negative Energy: -0.5009
Epoch   1 | Batch   20 | Positive Energy: -1.6297 | Negative Energy: -1.6208
Epoch   1 | Batch   30 | Positive Energy: -2.9502 | Negative Energy: -2.9290
Epoch   1 | Batch   40 | Positive Energy: -4.6229 | Negative Energy: -4.5868
Epoch   1 | Batch   50 | Positive Energy: -6.8295 | Negative Energy: -6.7748
Epoch   1 | Batch   60 | Positive Energy: -9.7547 | Negative Energy: -9.6765
Epoch   1 | Batch   70 | Positive Energy: -13.6310 | Negative Energy: -13.5232
Epoch   1 | Batch   80 | Positive Energy: -18.6865 | Negative Energy: -18.5427
Epoch   1 | Batch   90 | Positive Energy: -25.1460 | Negative Energy: -24.9607
Epoch   1 | Batch  100 | Positive Energy: -33.1723 | Negative Energy: -32.9417
Epoch   1 | Batch  110 | P

In [4]:
# Cell 4: Save Trained Weights + Loss History
print("\n" + "="*60)
print("Saving model weights and training history...")

# Create directories
os.makedirs('models', exist_ok=True)
os.makedirs('history', exist_ok=True)

# Save model weights
weights_path = 'models/energy_model_weights.h5'
cd_model.energy_net.save_weights(weights_path)
print(f"Model weights saved to: {weights_path}")

# Save full model architecture
model_path = 'models/energy_model_full.h5'
cd_model.energy_net.save(model_path)
print(f"Full model saved to: {model_path}")

# Save training history
history_path = 'history/training_history.pkl'
with open(history_path, 'wb') as f:
    pickle.dump(training_history, f)
print(f"Training history saved to: {history_path}")

# Print final statistics
print(f"\nFinal training statistics:")
print(f"  Final loss: {training_history['loss'][-1]:.6f}")
print(f"  Final positive energy: {training_history['pos_energy'][-1]:.4f}")
print(f"  Final negative energy: {training_history['neg_energy'][-1]:.4f}")
print(f"  Energy gap (pos - neg): {training_history['pos_energy'][-1] - training_history['neg_energy'][-1]:.4f}")

print("\n✓ All files saved successfully")


Saving model weights and training history...
Model weights saved to: models/energy_model_weights.h5


Full model saved to: models/energy_model_full.h5
Training history saved to: history/training_history.pkl

Final training statistics:
  Final loss: -0.953223
  Final positive energy: -250.3974
  Final negative energy: -249.4443
  Energy gap (pos - neg): -0.9532

✓ All files saved successfully
